In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel('../data/SPY.xlsx')
df.columns = df.iloc[5].values
df = df.iloc[6:].iloc[::-1].reset_index(drop=True)
df['Log Return'] = df.PX_LAST.apply(lambda x: np.log(x)).diff()
df.head()

In [ ]:
((np.array([1,2,3,4,5,6,7,8]) < np.ones(8) * 2) - 1) * ((np.array([1,2,3,4,5,6,7,8]) < np.ones(8) * 2) - 1)

In [ ]:
def symmetric(y, beta, q):
    """
    f_t(beta) = b1 + b2 * f_t-1(beta) + b3 * |y_t-1|
    """
    VaR = np.zeros_like(y)
    VaR[0] = np.quantile(y[:300], q)
    
    b1, b2, b3 = beta
    for i in range(1, len(y)):
        VaR[i] = b1 + b2 * VaR[i-1] + b3 * abs(y[i-1])
    
    return VaR

def loss(y, VaR, q):
    dev = y - VaR
    return np.mean(np.maximum(q * dev, (q - 1) * dev))

def sigmoid(x, G):
    """
    mimic indicator function I(x<=0)
    """
    return 1 / (1 + np.exp(G*x))

def derivatives_fbeta(model):
    """
    symmetric:
    asymmetric:
    adaptive:
    igarch:
    """
    if model == 'symmetric':
        pass # -1, - f[t-1]
    elif model == 'asymmetric':
        pass
    elif model == 'adaptive':
        pass
    elif model == 'igarch':
        pass
    else:
        raise ValueError('Wrong Model!')

def get_gradient(y, beta, VaR, q, G):
    """
      d(RQ)/dbeta
    = sum[ (q - sigmoid(y - f(beta))) * d(y - f(beta))/dbeta +
           (y - f(beta)) * d(q - sigmoid(y - f(beta)))/dbeta ]
    """
    gradient = np.zeros_like(beta)
    
    dev = y - VaR
    sigmoid_dev = sigmoid(dev, G)
    for t in range(1, len(y)):
        gradient[0] += (
            (q - sigmoid_dev[t]) * (- 1 - beta[1] * gradient[0]) +
            dev[t] * ((sigmoid_dev[t] - sigmoid_dev[t] ** 2) * G * (- 1 - beta[1] * gradient[0]))
        )
        
        gradient[1] += (
            (q - sigmoid_dev[t]) * (- beta[1] * gradient[1] - VaR[t-1]) +
            dev[t] * ((sigmoid_dev[t] - sigmoid_dev[t] ** 2) * G * (- VaR[t-1] - beta[1] * gradient[1]))
        )
        
        gradient[2] += (
            (q - sigmoid_dev[t]) * (- abs(y[t-1]) - beta[1] * gradient[2]) +
            dev[t] * ((sigmoid_dev[t] - sigmoid_dev[t] ** 2) * G * (- abs(y[t-1]) - beta[1] * gradient[2]))
        )
    return gradient

def gradient_descent(y, q=0.05, G=10, learning_rate=1e-10, max_iter=100, tol=1e-10):
    y = np.array(y)
    beta = np.random.uniform(-1, 1, 3)
    VaR = symmetric(y, beta, q)
    last = loss(y, VaR, q)
    print(f'[0/{max_iter}] Loss: {last:.4f}')
    
    for i in range(max_iter):
        gradient = get_gradient(y, beta, VaR, q, G)
        beta -= learning_rate * gradient
        VaR = symmetric(y, beta, q)
        current = loss(y, VaR, q)
        print(f'[{i+1}/{max_iter}] Loss: {current:.4f} Beta: {beta}')
        if last - current < -np.inf: # tol:
            print('Early stopped.')
            print(f'Final Loss: {current:.4f} Beta: {beta}')
            return beta + learning_rate * gradient
        
        last = current
    
    print(f'Finished {max_iter} loop')
    print(f'Final Loss: {current:.4f}')
    return beta

In [ ]:
returns = (df['Log Return'].dropna() - df['Log Return'].dropna().mean()).reset_index(drop=True) * 100

In [ ]:
beta = gradient_descent(returns)

In [ ]:
VaR = symmetric(returns, beta, q=0.05)
plt.plot(returns)
plt.plot(VaR)
np.sum(returns < VaR) / len(returns)